In [1]:
from model import aysennet_model
from tensorflow.keras.callbacks import History 


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 128)     3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 300, 128)     512       
_________________________________________________________________
activation (Activation)      (None, 300, 300, 128)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 256)     295168    
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 256)     1024      
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 256)     0         
__________

In [2]:
from tensorflow.keras.optimizers import Adam
awesome_model = aysennet_model()
awesome_model.summary()
metrics_x=['accuracy']

adam = Adam(lr=0.000074)
awesome_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam,metrics=metrics_x)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 300, 300, 128)     3584      
_________________________________________________________________
batch_normalization_15 (Batc (None, 300, 300, 128)     512       
_________________________________________________________________
activation_15 (Activation)   (None, 300, 300, 128)     0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 150, 150, 128)     0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 150, 150, 256)     295168    
_________________________________________________________________
batch_normalization_16 (Batc (None, 150, 150, 256)     1024      
_________________________________________________________________
activation_16 (Activation)   (None, 150, 150, 256)     0         
__________

In [3]:
from tensorflow.keras.callbacks import History 
from tensorflow.keras.callbacks import Callback

def generate_train_data():
    while True:
        for x in range(0,2000):
            # create numpy arrays of input data
            # and labels, from each line in the file
            x_train, y_train = load_batch_data(x,"train")
            #print(x_train.shape)
            yield (x_train,y_train)

def generate_validation_data():
    while True:
        for x in range(0,300):
            # create numpy arrays of input data
            # and labels, from each line in the file
            x_train, y_train = load_batch_data(x,"validation")
            yield (x_train,y_train)
        
#history = History()     
#generator_train = generate_train_data()
#generator_val = generate_validation_data()


In [5]:
import os
import cv2
import numpy as np
dataset_path = "../data"

base = 0
labels = []


def load_labels():
    return os.listdir(dataset_path)


max_train_data = 30000
max_test_data = 10000
max_val_data = 10000

sample_size_train = 7200
sample_size_val = 1920


def load_batch_data_test(start_point, whichData="test"):
    labels = load_labels()
    labels.sort()
    # print(labels)

    data_step = 0

    if "train" == whichData:
        dataset = np.ndarray(shape=(sample_size_train, 300, 300, 3),
                             dtype=np.float32)
        dataset_labels = np.zeros(sample_size_train, dtype=np.int32)
        #print(dataset_labels.shape)
        data_step = 600
    elif "validation" == whichData:
        dataset = np.ndarray(shape=(sample_size_val, 300, 300, 3),
                             dtype=np.float32)
        dataset_labels = np.zeros(sample_size_val,dtype=np.int32)
        data_step = 160
    elif "test" == whichData:
        dataset = np.ndarray(shape=(sample_size_val, 300, 300, 3),
                             dtype=np.float32)
        dataset_labels = np.zeros(sample_size_val, dtype=np.int32)
        data_step = 100

    count = 0
    label_int = 0
    for label in labels:
        temp = os.listdir(dataset_path + "/" + label + "/" + whichData + "/patches/")
        # print(temp)

        for x in range(start_point, start_point + data_step):
            img = cv2.imread(dataset_path + "/" + label + "/" + whichData + "/patches/" + temp[x])
            # print(img)
            dataset[count] = img
            dataset_labels[count] = label_int
            #print(dataset_labels[label_int])
            count += 1
        label_int += 1

    return dataset, dataset_labels

#x_train, y_train = load_batch_data_test(0,"train")

#print(y_train.shape)


In [6]:
# serialize model to JSON
def saveModelToDisk(x,model):
    model_json = model.to_json()
    with open("awesome_model-6.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("checkpoints/awesome_model-6"+str(x)+".h5")
    print("Saving awesome model to disk")

    
for x in range(0, 50):
    print("RANGE")
    print(x)
    x_train, y_train = load_batch_data_test(x,"train")
    x_val, y_val = load_batch_data_test(x,"validation")
    #validation_data=(x_val, y_val)
   
    awesome_model.fit(x=x_train, y=y_train, batch_size=20, epochs=2, verbose=1, shuffle=True,validation_data=(x_val, y_val))
    #awesome_model.fit(x=x_train, y=y_train, batch_size=100, epochs=10, verbose=1, shuffle=True,validation_split=0.2)

    if x % 10 == 0:
        saveModelToDisk(x,awesome_model)

#result = awesome_model.fit(x=x_train, y=y_train, batch_size=100, epochs=1, verbose=1, shuffle=True)
#


#result = awesome_model.fit_generator(generator_train,steps_per_epoch=2000,epochs=1000 ,validation_data=generator_val,validation_steps=300, verbose=1, callbacks=[history],initial_epoch=0)

RANGE
0
Train on 7200 samples, validate on 1920 samples
Epoch 1/2
7200/7200 [==============================] - 172s 24ms/step - loss: 2.1074 - acc: 0.2997 - val_loss: 2.1184 - val_acc: 0.2781
Epoch 2/2
7200/7200 [==============================] - 166s 23ms/step - loss: 1.8075 - acc: 0.4249 - val_loss: 1.9491 - val_acc: 0.3542
Saving awesome model to disk
RANGE
1
Train on 7200 samples, validate on 1920 samples
Epoch 1/2
7200/7200 [==============================] - 164s 23ms/step - loss: 1.6877 - acc: 0.4724 - val_loss: 1.7883 - val_acc: 0.4167
Epoch 2/2
7200/7200 [==============================] - 165s 23ms/step - loss: 1.6067 - acc: 0.5022 - val_loss: 1.7891 - val_acc: 0.4318
RANGE
2
Train on 7200 samples, validate on 1920 samples
Epoch 1/2
7200/7200 [==============================] - 166s 23ms/step - loss: 1.5378 - acc: 0.5346 - val_loss: 1.7984 - val_acc: 0.4297
Epoch 2/2
7200/7200 [==============================] - 166s 23ms/step - loss: 1.4642 - acc: 0.5651 - val_loss: 1.7386 - val

KeyboardInterrupt: 

In [7]:
result = awesome_model.fit(x=x_train, y=y_train, batch_size=20, epochs=1, verbose=1, shuffle=True,validation_data=(x_val, y_val))

saveModelToDisk(999,awesome_model)

#result = awesome_model.fit(x=x_train, y=y_train, batch_size=100, epochs=1, verbose=1, shuffle=True,validation_split=0.2)


print(result.history.keys())
print(result.history["loss"])
print(result.history["acc"])

Train on 7200 samples, validate on 1920 samples
Epoch 1/1
7200/7200 [==============================] - 167s 23ms/step - loss: 0.5610 - acc: 0.8799 - val_loss: 1.6171 - val_acc: 0.5573
Saving awesome model to disk
dict_keys(['acc', 'val_acc', 'val_loss', 'loss'])
[0.5609642696049478]
[0.8798611094554265]


In [8]:
X,Y = load_batch_data_test(55,"test")
scores = awesome_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (awesome_model.metrics_names[1], scores[1]*100))

from tensorflow.keras.models import model_from_json

acc: 29.11%


In [9]:
Y_NEW = awesome_model.predict(X)

for i in range(len(X)):
     print("Y=%s, Predicted=%s" % (Y[i], Y_NEW[i], ))

Y=0, Predicted=[0.03228959 0.00982952 0.01666539 0.01599968 0.00734754 0.01051196
 0.02056134 0.78723097 0.00747707 0.01350144 0.03988748 0.03869808]
Y=0, Predicted=[0.01654285 0.01165858 0.03131125 0.09450554 0.02766298 0.02442855
 0.08693046 0.0435086  0.04472722 0.03035761 0.5687137  0.01965267]
Y=0, Predicted=[0.00768709 0.02012967 0.02172349 0.01112148 0.05018903 0.01193305
 0.066734   0.00992733 0.0473838  0.01769313 0.03290673 0.7025712 ]
Y=0, Predicted=[0.03371592 0.05877891 0.04282996 0.0942812  0.01998052 0.08067609
 0.37764993 0.08281138 0.03805001 0.02393805 0.06219517 0.08509284]
Y=0, Predicted=[0.00470932 0.0114006  0.01304071 0.01086143 0.00622947 0.83204997
 0.00962693 0.01586807 0.01013919 0.00762784 0.0671697  0.01127669]
Y=0, Predicted=[0.09477501 0.01625218 0.03026861 0.05099697 0.019265   0.01977767
 0.02160655 0.02381419 0.579112   0.02286879 0.08671979 0.03454328]
Y=0, Predicted=[0.00467915 0.0102454  0.009135   0.02577941 0.0160665  0.00684381
 0.64502    0.0080

 0.14070861 0.05251334 0.09447152 0.02203342 0.15052636 0.09254717]
Y=8, Predicted=[0.00576615 0.01151234 0.00603931 0.02918507 0.00571858 0.01412147
 0.01218444 0.01722284 0.8753895  0.00668937 0.00847863 0.0076924 ]
Y=8, Predicted=[0.02386838 0.0349023  0.04659635 0.2248437  0.01907027 0.01458984
 0.10515309 0.33935666 0.02298643 0.02325764 0.11133906 0.03403632]
Y=8, Predicted=[0.00511809 0.02082551 0.00601431 0.0259166  0.00624829 0.00732672
 0.01041927 0.00581969 0.8611399  0.00790146 0.03177806 0.01149221]
Y=8, Predicted=[0.01965306 0.02046171 0.02301608 0.04503414 0.2803951  0.14575802
 0.03479053 0.03633768 0.03046355 0.01758439 0.27683204 0.06967375]
Y=8, Predicted=[0.00779369 0.01813782 0.03724708 0.04314558 0.03185673 0.01524375
 0.02338448 0.02825779 0.24141617 0.01795373 0.4941594  0.04140372]
Y=8, Predicted=[0.00926068 0.02788047 0.01396729 0.42307177 0.02806554 0.02141179
 0.05920608 0.01588707 0.22376591 0.07408311 0.0725985  0.03080174]
Y=8, Predicted=[0.00314293 0.006

 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.00353824 0.00801469 0.02966463 0.01677494 0.00409065
 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.00353824 0.00801469 0.02966463 0.01677494 0.00409065
 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.00353824 0.00801469 0.02966463 0.01677494 0.00409065
 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.00353824 0.00801469 0.02966463 0.01677494 0.00409065
 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.00353824 0.00801469 0.02966463 0.01677494 0.00409065
 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.00353824 0.00801469 0.02966463 0.01677494 0.00409065
 0.0132396  0.00565311 0.0047084  0.00393502 0.8610513  0.0398462 ]
Y=0, Predicted=[0.00948317 0.003